In [6]:
import numpy as np
import onnx
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/tmp/quartznet.onnx")
count=0
for n in model.graph.node:
    count += 1
print(count)

931


In [5]:
import numpy as np
import onnx
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
from finn.transformation.general import GiveRandomTensorNames, GiveReadableTensorNames

Conv1_node = onnx.helper.make_node(
    "Conv",
    inputs=['in1_conv1', 'in2_conv1'],
    outputs=['out1_conv1'],
    name="Conv1",
    dilations=[2],
    group=512,
    kernel_shape=[87],
    pads=[86, 86],
    strides=[1]
)

in1_conv1 = onnx.helper.make_tensor_value_info("in1_conv1", onnx.TensorProto.FLOAT, [1, 512, 128])
in2_conv1 = onnx.helper.make_tensor_value_info("in2_conv1", onnx.TensorProto.FLOAT, [512, 1, 87])
out1_conv1 = onnx.helper.make_tensor_value_info("out1_conv1", onnx.TensorProto.FLOAT, [1, 512, 128])

graph = onnx.helper.make_graph(
    nodes=[Conv1_node],
    name="test_graph",
    inputs=[in1_conv1],
    outputs=[out1_conv1],
    value_info=[in2_conv1]
)

def set_all_initializers(model, value):
    """ Sets weight values of the conv to 'value'. """
    for n in model.graph.node:
        if len(n.input) > 1:
            init_name = n.input[1]
            model.set_initializer(init_name, value)

onnx_model = onnx.helper.make_model(graph, producer_name="test_graph-model")
model = ModelWrapper(onnx_model)

# Initialize appropriately
model.set_tensor_datatype('in2_conv1', DataType.INT4)        
weight_values = np.load('/tmp/weight_values.npy')
set_all_initializers(model, weight_values)

model = model.transform(GiveRandomTensorNames())
model = model.transform(GiveReadableTensorNames())
model.save("/tmp/test_model_dilations.onnx")

showInNetron("/tmp/test_model_dilations.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/test_model_dilations.onnx' at http://0.0.0.0:8081


In [4]:
from finn.transformation.change_3d_tensors_to_4d import Change3DTo4DTensors
from finn.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from finn.transformation.general import GiveUniqueNodeNames, GiveRandomTensorNames, GiveReadableTensorNames

model = ModelWrapper("/tmp/test_model_dilations.onnx")
model = model.transform(Change3DTo4DTensors())
model = model.transform(LowerConvsToMatMul())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveRandomTensorNames())
model = model.transform(GiveReadableTensorNames())
model.save("/tmp/test_model_dilations_lowered.onnx")

showInNetron("/tmp/test_model_dilations_lowered.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/test_model_dilations_lowered.onnx' at http://0.0.0.0:8081


In [20]:
model = ModelWrapper("/tmp/test_model_dilations_lowered.onnx")

for n in model.graph.node:
    if n.op_type=='MatMul':
        print(n)
        weight_im2col = model.get_initializer("MatMul_0_param0")
print(type(weight_im2col))

np.save("/tmp/weight_values_matmul.npy", weight_im2col)

input: "Im2Col_0_out0"
input: "MatMul_0_param0"
output: "MatMul_0_out0"
name: "MatMul_0"
op_type: "MatMul"

<class 'numpy.ndarray'>


# Testing and obtaining output values

In [3]:
import finn.core.onnx_exec as oxe
from finn.util.basic import gen_finn_dt_tensor
import random

def generate_random_input(model, randomize, value=[]):
        """ Creates input dictionary with a random numpy array that matches the input tensor shape """
        i_shape = []
        input_dict={}
        
        if randomize is True:
            for i in range(len(model.graph.input)):
                input_node = model.graph.input[i]
                input_node_name = input_node.name
                input_node_shape = model.get_tensor_shape(input_node_name)
                i_val = gen_finn_dt_tensor(DataType.FLOAT32, input_node_shape)
                input_dict[input_node_name] = i_val
            return input_dict
        else:
            for i in range(len(model.graph.input)):
                input_node = model.graph.input[i]
                input_node_name = input_node.name
                input_dict[input_node_name] = value
            return input_dict

        
global_in = []
global_out = []
im2col_in = []
im2col_out = []
model1 = ModelWrapper("/tmp/test_model_dilations.onnx")
model2 = ModelWrapper("/tmp/test_model_dilations_lowered.onnx")

original_out = []

for i in range(10):
    
    in_val1 = np.random.randint(-1000,1000,[1,512,128]).astype(np.float32)    
    in_val2 = np.reshape(in_val1, np.shape(in_val1)+(1,))
    
    input_dict1 = generate_random_input(model1, False, in_val1)
    input_dict2 = generate_random_input(model2, False, in_val2)
    
    output_node_name1 = model1.graph.output[0].name
    output_node_name2 = model2.graph.output[0].name
    
    output_dict1 = oxe.execute_onnx(model1, input_dict1, return_full_exec_context=True)
    output_dict2 = oxe.execute_onnx(model2, input_dict2, return_full_exec_context=True)
    
    expected1 = output_dict1[output_node_name1]
    expected2 = output_dict2[output_node_name2]
        
    global_in.append(in_val2)
    global_out.append(expected2)
    im2col_in.append(output_dict2["Transpose_0_out0"])
    im2col_out.append(output_dict2["Im2Col_0_out0"])
    
    original_out.append(expected1)
    

np.save("/tmp/global_in.npy", global_in)
np.save("/tmp/global_out.npy", global_out)
np.save("/tmp/im2col_in.npy", im2col_in)
np.save("/tmp/im2col_out.npy", im2col_out)
np.save("/tmp/original_out.npy", original_out)



In [15]:
original_output = np.load("/tmp/original_out.npy")
lowered_output = np.load("/tmp/global_out.npy")

for i in range(np.shape(original_output)[0]):
    out1 = original_output[i]
    out2 = lowered_output[i]
    
    out1 = np.reshape(out1, np.shape(out2))
    
    print(np.array_equal(out1,out2))
    
np.save("/tmp/global_out.npy",lowered_output2)

True
True
True
True
True
True
True
True
True
True


In [61]:
input_dict1_val = input_dict1['in1_conv1']
input_dict2_val = input_dict2['global_in']
input_dict2_val = np.reshape(input_dict2_val, np.shape(input_dict1_val))

is_eq = np.array_equal(input_dict1_val, input_dict2_val)
print(is_eq)

expected2 = np.load("/tmp/test_output_values.npy")
print(np.shape(expected2))
expected2 = expected2[0]
expected1 = np.reshape(expected1, np.shape(expected2))

print("{}\t{}".format(np.shape(expected1),np.shape(expected2)))
is_eq = np.array_equal(expected1, expected2)
print(is_eq)

print(np.shape(expected2))

print("{:.10f}".format(expected1[0][0][0][0]))
print("{:.10f}".format(expected2[0][0][0][0]))

for k,v in output_dict2.items():
    print("{}\t{}".format(k,np.shape(v)))


True
(3, 1, 512, 128, 1)
(1, 512, 128, 1)	(1, 512, 128, 1)
True
(1, 512, 128, 1)
862.0000000000
862.0000000000
global_in	(1, 512, 128, 1)
global_out	(1, 512, 128, 1)
MatMul_0_param0	(44544, 512)
Transpose_0_out0	(1, 128, 1, 512)
Im2Col_0_out0	(1, 128, 1, 44544)
MatMul_0_out0	(1, 128, 1, 512)
